# Subtractor
Quantum computers have the function of being able to perform the same calculations as conventional computers. Check the circuit for Subtractor.

## What we'll learn this time
1. Implementing Binary Subtractor Using Quantum Gates
2. Using quantum superposition, we can realize multiple Substractor in a single circuit

## Install Blueqat
Install Blueqat from pip.

In [1]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## Subtraction of binary numbers
Subtractor uses a CCX gate to perform minus sign or a plus sign and a CX gate to perform Subtraction. In this case, we will do a quantum circuit of binary Subtraction, a-b = cd. This time, we will implement four types of Subtraction according to the values of a and b. Each Subtraction is,

0-0 = 00 => 0000  
0-1 = 11 => 0111  
1-0 = 01 => 1001  
1-1 = 00 => 1100  

The first two qubits are the input values a and b, and the second two qubits are the output values c and d. I've built a separate circuit for the input of a and b and a circuit for the addition, and used it several times. For data input, use the X gate to flip 0 to 1.

The circuit part of the additon looks like this. * is a controlled bit.

```
a ---X---*---X---*------- a
b -------*-------|---*--- b
0 -------X-------|---|--- c
0 ---------------X---X--- d
```

In [5]:
#import tools
from blueqat import Circuit

#subtraction part
subtractor = Circuit().x[0].ccx[0,1,2].x[0].cx[0,3].cx[1,3]

In [6]:
#0-0
(Circuit() + subtractor).m[:].run(shots=100)

Counter({'0000': 100})

In [7]:
#0-1
(Circuit().x[1] + subtractor).m[:].run(shots=100)

Counter({'0111': 100})

In [8]:
#1-0
(Circuit().x[0] + subtractor).m[:].run(shots=100)

Counter({'1001': 100})

In [9]:
#1-1
(Circuit().x[0,1] + subtractor).m[:].run(shots=100)

Counter({'1100': 100})

As you can see, we were able to calculate.

## subtraction using superposition
Now, instead of filling in the data one by one with the X-gate, let's use the H-gate to do the subtraction.

In [10]:
#Use H gate as input instead of X gate
(Circuit().h[0,1] + subtractor).m[:].run(shots=100)

Counter({'1001': 18, '0000': 21, '1100': 36, '0111': 25})

Using the Hadamal Gate, I got 4 answers, about 1/4 each. If you create a universal subtraction circuit like this, you can perform calculations using the superposition state.

## subtraction using entanglement
Next, instead of the H-gate, we try to subtract a-b=0 using quantum entanglement.

In [11]:
#Make an entangled state of 00 and 11
(Circuit().h[0].cx[0,1] + subtractor).m[:].run(shots=100)

Counter({'1100': 51, '0000': 49})

Since the input values 00 and 11 are entangled, the addition of these two will come out about half by half.

--------